In [1]:
import pandas as pd
import numpy as np

import data_process
import importlib
importlib.reload(data_process)

<module 'data_process' from 'c:\\Users\\panto\\Documents\\git\\personal\\EV_energy_comm\\python\\data_process.py'>

In [2]:
# create random time series from import data
da = data_process.DataAction()
nc = data_process.net_calc()

da.folder_path = r"G:\\My Drive\\docs\\Education\\University\\ALUF\\SSE\\6 sem\\Bachelor Thesis\\data\\"
keep_cols = ['DE_KN_residential1_grid_import',
             'DE_KN_residential2_grid_import',
             'utc_timestamp']
da.imp_procc("house_data.csv", keep_cols)

dfList created successfully.


In [3]:
# used dataset size
pd.to_datetime(da.dfList[88].iloc[-1].name)-pd.to_datetime(da.dfList[0].iloc[1].name)

Timedelta('618 days 01:18:00')

In [4]:
# broaden loc times
da.night_evening_t = '18:00:00'
da.night_morning_t = '06:00:00'

In [5]:
def night_list(dfList_val):
    """create df list of nights for single dfList list"""

    ts_nights = []
    for i in da.unique_date(dfList_val)[:-1]: # last date has no morning time, is dropped
        night = da.get_night(dfList_val,i)
        ts_nights.append(night)

    return ts_nights

### Maxes

In [44]:
def max_datetimes_concat(ts_nights):
    """write max times from ts_night to max_datetimes df"""

    concat_list = []
    for i in range(len(ts_nights)):
        concat_list.append(pd.Series(ts_nights[i].idxmax()))

    max_datetimes=pd.concat((concat_list), axis=1).transpose()

    return max_datetimes

In [45]:
max_datetimes = pd.DataFrame()
parse_max = []

for i in range(len(da.dfList)):
    ts = da.parse_procc(da.dfList[i]).rolling(10).sum()
    ts_nights = night_list(ts)
    parse_max.append(max_datetimes_concat(ts_nights))

max_datetimes = pd.concat((parse_max))
max_datetimes.head()

,DE_KN_residential1_grid_import,DE_KN_residential2_grid_import
0,2015-05-21 19:06:00+02:00,2015-05-21 18:04:00+02:00
1,2015-05-22 20:09:00+02:00,2015-05-22 20:46:00+02:00
2,2015-05-23 20:54:00+02:00,2015-05-23 21:25:00+02:00
3,2015-05-24 22:45:00+02:00,2015-05-25 00:54:00+02:00
4,2015-05-25 21:57:00+02:00,2015-05-25 21:00:00+02:00


In [46]:
print("Global max load time mean:")
print(np.mean(max_datetimes.max()).strftime("%H:%M:%S"))
max_datetimes.describe(datetime_is_numeric=True)

Global max load time mean:
19:23:00


,DE_KN_residential1_grid_import,DE_KN_residential2_grid_import
count,622,622
mean,2016-03-27 09:12:15.916398592+02:00,2016-03-27 08:39:41.575562752+02:00
min,2015-05-21 19:06:00+02:00,2015-05-21 18:04:00+02:00
25%,2015-10-24 02:14:30+02:00,2015-10-24 02:57:00+02:00
50%,2016-03-27 09:08:30+02:00,2016-03-27 09:36:30+02:00
75%,2016-08-29 14:48:30+02:00,2016-08-29 12:57:00+02:00
max,2017-01-31 19:51:00+01:00,2017-01-31 18:55:00+01:00


### Mins

In [47]:
def min_datetimes_concat(ts_nights):
    """write min times from ts_night to min_datetimes df"""

    concat_list = []
    for i in range(len(ts_nights)):
        concat_list.append(pd.Series(ts_nights[i].idxmin()))

    min_datetimes=pd.concat((concat_list), axis=1).transpose()

    return min_datetimes

In [48]:
min_datetimes = pd.DataFrame()
parse_min = []

for i in range(len(da.dfList)):
    ts = da.parse_procc(da.dfList[i]).rolling(10).sum()
    ts_nights = night_list(ts)
    parse_min.append(min_datetimes_concat(ts_nights))

min_datetimes = pd.concat((parse_min))
min_datetimes.head()

,DE_KN_residential1_grid_import,DE_KN_residential2_grid_import
0,2015-05-22 03:11:00+02:00,2015-05-22 01:41:00+02:00
1,2015-05-22 20:49:00+02:00,2015-05-22 22:11:00+02:00
2,2015-05-24 03:10:00+02:00,2015-05-23 18:02:00+02:00
3,2015-05-24 20:14:00+02:00,2015-05-24 18:26:00+02:00
4,2015-05-26 01:26:00+02:00,2015-05-25 18:13:00+02:00


In [49]:
print("Global min load time mean:")
print(np.mean(min_datetimes.min()).strftime("%H:%M:%S"))
min_datetimes.describe(datetime_is_numeric=True)

Global min load time mean:
02:26:00


,DE_KN_residential1_grid_import,DE_KN_residential2_grid_import
count,622,622
mean,2016-03-27 13:44:23.633440512+02:00,2016-03-27 12:52:11.575562496+02:00
min,2015-05-22 03:11:00+02:00,2015-05-22 01:41:00+02:00
25%,2015-10-24 11:20:00+02:00,2015-10-24 06:03:30+02:00
50%,2016-03-27 16:02:30+02:00,2016-03-27 13:16:30+02:00
75%,2016-08-29 19:09:45+02:00,2016-08-29 17:29:45+02:00
max,2017-02-01 01:04:00+01:00,2017-02-01 01:36:00+01:00
